In [1]:
# =========================
# 0) 安裝與匯入
# =========================
!nvidia-smi -L || true
!pip -q install torch torchvision timm

import os, json, csv
from pathlib import Path
from datetime import datetime

import torch
import torch.nn as nn
import timm

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


GPU 0: NVIDIA L4 (UUID: GPU-6daa7290-e6eb-a876-b091-fbaa7405e361)
Mounted at /content/drive


In [2]:
# =========================
# 1) 基本參數（請改成你的路徑）
# =========================

# 你的已訓練權重
WEIGHT_PATH = "/content/drive/MyDrive/Colab_Results/leaf_stage1_cls/20251204/best_mobilenetv3_large.pth"

# 要預測的單張圖片路徑（可先把圖放進 Drive）
IMG_PATH = "/content/drive/MyDrive/samples/2349.jpg"

# 結果存回雲端的位置
SAVE_TO_DRIVE_DIR = "/content/drive/MyDrive/Colab_Results/leaf_stage1_cls_single_infer"

# 與你訓練時一致
MODEL_NAME = "mobilenetv3_large_100"
CLASS_NAMES = ['others','pepper_bell','potato','tomato','whole_plant']  # 5 類
IMG_SIZE = 224
TOPK = 5  # 顯示前 K 名

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Path(SAVE_TO_DRIVE_DIR).mkdir(parents=True, exist_ok=True)
RUN_DIR = Path(SAVE_TO_DRIVE_DIR) / datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR.mkdir(parents=True, exist_ok=True)

print("device:", DEVICE)
print("classes:", CLASS_NAMES)
print("save to:", RUN_DIR)


device: cuda
classes: ['others', 'pepper_bell', 'potato', 'tomato', 'whole_plant']
save to: /content/drive/MyDrive/Colab_Results/leaf_stage1_cls_single_infer/20251212_092825


In [3]:
# =========================
# 2) 建立模型並載入權重（不需重訓）
# =========================
num_classes = len(CLASS_NAMES)

model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=num_classes)
ckpt = torch.load(WEIGHT_PATH, map_location="cpu")
model.load_state_dict(ckpt, strict=True)
model.to(DEVICE)
model.eval()

print(f"[OK] Loaded weights: {WEIGHT_PATH}")


[OK] Loaded weights: /content/drive/MyDrive/Colab_Results/leaf_stage1_cls/20251204/best_mobilenetv3_large.pth


In [4]:
# =========================
# 3) 前處理（與訓練一致）
# =========================
preprocess = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

def load_image(path: str):
    img = Image.open(path).convert("RGB")
    return img


In [5]:
# =========================
# 4) 單張圖片推論函式
# =========================
@torch.no_grad()
def predict_one(image_path: str, topk: int = 5):
    img = load_image(image_path)
    x = preprocess(img).unsqueeze(0).to(DEVICE)  # [1, C, H, W]
    logits = model(x)
    probs = logits.softmax(1).squeeze(0).cpu().numpy()  # [num_classes]

    # 取 top-k
    k = min(topk, len(CLASS_NAMES))
    idx = np.argsort(-probs)[:k]
    top_labels = [CLASS_NAMES[i] for i in idx]
    top_probs  = probs[idx]
    pred_idx = int(np.argmax(probs))
    pred_label = CLASS_NAMES[pred_idx]
    pred_prob  = float(probs[pred_idx])

    return {
        "image_path": image_path,
        "pred_label": pred_label,
        "pred_prob": pred_prob,
        "all_probs": {CLASS_NAMES[i]: float(probs[i]) for i in range(len(CLASS_NAMES))},
        "topk": [{"label": l, "prob": float(p)} for l, p in zip(top_labels, top_probs)]
    }


In [6]:
# =========================
# 5) 執行單張推論 & 顯示/儲存結果
# =========================
res = predict_one(IMG_PATH, topk=TOPK)

# --- 文字輸出 ---
print(f"\n[Prediction]")
print(f"Image     : {res['image_path']}")
print(f"Pred label: {res['pred_label']}  (confidence={res['pred_prob']:.4f})")
print("\nTop-K:")
for r in res["topk"]:
    print(f" - {r['label']:<12}  {r['prob']:.4f}")

# --- 視覺化：原圖 + 機率長條圖 ---
img = load_image(IMG_PATH)

plt.figure(figsize=(11,4))
plt.subplot(1,2,1)
plt.imshow(img)
plt.axis("off")
plt.title(f"Pred: {res['pred_label']} ({res['pred_prob']:.2%})")

plt.subplot(1,2,2)
labels = list(res["all_probs"].keys())
vals   = list(res["all_probs"].values())
ypos   = np.arange(len(labels))
plt.barh(ypos, vals)
plt.gca().set_yticks(ypos, labels)
plt.gca().invert_yaxis()
plt.xlabel("Probability")
plt.xlim(0,1)
plt.title("Class Probabilities")
plt.tight_layout()

fig_path = RUN_DIR / "prediction_plot.png"
plt.savefig(fig_path, dpi=160); plt.close()
print("[OK] saved plot ->", fig_path)

# --- 存 JSON & CSV ---
json_path = RUN_DIR / "result.json"
with open(json_path, "w") as f:
    json.dump(res, f, indent=2, ensure_ascii=False)
print("[OK] saved json ->", json_path)

csv_path = RUN_DIR / "prediction.csv"
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    header = ["image_path", "pred_label", "pred_prob"] + [f"prob_{c}" for c in CLASS_NAMES]
    writer.writerow(header)
    row = [res["image_path"], res["pred_label"], res["pred_prob"]] + [res["all_probs"][c] for c in CLASS_NAMES]
    writer.writerow(row)
print("[OK] saved csv  ->", csv_path)



[Prediction]
Image     : /content/drive/MyDrive/samples/2349.jpg
Pred label: whole_plant  (confidence=0.9851)

Top-K:
 - whole_plant   0.9851
 - potato        0.0073
 - tomato        0.0042
 - pepper_bell   0.0026
 - others        0.0009
[OK] saved plot -> /content/drive/MyDrive/Colab_Results/leaf_stage1_cls_single_infer/20251212_092825/prediction_plot.png
[OK] saved json -> /content/drive/MyDrive/Colab_Results/leaf_stage1_cls_single_infer/20251212_092825/result.json
[OK] saved csv  -> /content/drive/MyDrive/Colab_Results/leaf_stage1_cls_single_infer/20251212_092825/prediction.csv
